## Setup Code

In [22]:
import numpy as np
import math
from time import perf_counter
from collections import deque


class Node:
    def __init__(self, data, left=None, right=None, parent=None, depth:int=None):
        self.left = left
        self.data = data
        self.right = right
        self.parent = parent
        self.depth = depth

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

# def as_btree_nodes(node_vals: list[int]) -> TreeNode:
#     # breadth first order,left to right
#     root = TreeNode(node_vals[0])
#     q = deque([root])
#     node_vals.pop()
#     while node_vals:
#         node = q.pop()
#         nval = node_vals.pop()
#         if nval is not None:
#             node.left = TreeNode(nval)
#             q.appendleft(node.left)
#         if node_vals:
#             nval = node_vals.pop()
#             if nval is not None:
#                 node.right = TreeNode(nval)
#                 q.appendleft(node.right)

#     return root

def run_tests(fnc, test_cases) -> None:
    for i, case in enumerate(test_cases):
        ans = fnc(*case["in"])
        if ans != case["out"]:
            print(f"Test # {i+1} : Calculated {ans}, expected {case['out']}")

# In Progress

You are given the root of a binary tree.

A ZigZag path for a binary tree is defined as follow:

- Choose any node in the binary tree and a direction (right or left).
- If the current direction is right, move to the right child of the current node; otherwise, move to the left child.
- Change the direction from right to left or from left to right.
- Repeat the second and third steps until you can't move in the tree.


Zigzag length is defined as the number of nodes visited - 1. (A single node has a length of 0).

Return the longest ZigZag path contained in that tree.


- must alternate directions after each node
- max length
- will be at least 1 node
- DOES not necessarily include the root node

design
- depth first search
- what is the state - (node, direction, depth)


can store depth of zigzag branch

Q <- root, left, 0
Q <- root, right, 0
mx, depths = 0, {}
while Q:
    node, zag_dir, depth = Q.pop()
    if node.zag_dir:
        Q <- node.zag_dir, not zag_dir, depth + 1
    else:
        mx = max(mx, depth)

    if node.NOT_zag_dir:  # not in zag path, but potentially the head of its own path
        Q <- node.NOT_zag_dir, left, 0
        Q <- node.NOT_zag_dir, right, 0

return mx





In [ ]:
def zag_kids(node, is_left: bool) -> tuple[TreeNode|None]:
    if is_left:
        return node.left, node.right
    return node.right, node.left

def longest_zigzag(root: TreeNode) -> int:
    stack, _max = [], 0
    stack.append((root, True, 0))  # left
    stack.append((root, False, 0))

    while stack:
        node, is_left, depth = stack.pop()
        zag, no_zag = zag_kids(node, is_left)
        if zag:
            stack.append((zag, not is_left, depth+1))
        else:
            _max = max(_max, depth)
        
        if no_zag:
            stack.append((no_zag,  False, 0))
            stack.append((no_zag, True, 0))
    return _max

In [ ]:
def longest_zigzag(root: TreeNode) -> int:
    stack, _max = [], 0
    stack.append((root, "left", 0))
    stack.append((root, "right", 0))

    while stack:
        node, _dir, depth = stack.pop()
        not_dir = "left" if _dir is "right" else "right"
        if child := getattr(node, _dir):
            stack.append((child, not_dir, depth+1))
        else:
            _max = max(_max, depth)
        
        if child := getattr(node, not_dir):
            stack.append((child, "left", 0))
            stack.append((child, "right", 0))
    return _max

# visited will be the depth of the branch under
# (node_id, direction) -> depth

# def longest_zigzag(root: TreeNode) -> int:
#     Q, _max, visited = deque(), 0, {}
#     Q.appendleft((root, "left", 0))
#     Q.appendleft((root, "right", 0))

#     while Q:
#         node, _dir, depth = Q.pop()
#         not_dir = "left" if _dir is "right" else "right"
#         if child := getattr(node, _dir):
#             if d := visited.get((id(child), not_dir)):
#                 _max = max(_max, depth + d)
#             else:
#                 Q.appendleft((child, not_dir, depth+1))
#         else:
#             visited[(id(child), not_dir)] = depth
#             _max = max(_max, depth)
        
#         if child := getattr(node, not_dir):
#             Q.appendleft((child, "left", 0))
#             Q.appendleft((child, "right", 0))
        
#     return _max

Given the root of a binary tree and an integer targetSum, return the number of paths where the sum of the values along the path equals targetSum.

The path does not need to start or end at the root or a leaf, but it must go downwards (i.e., traveling only from parent nodes to child nodes).

Restate the problem
Identify components
number of (partial) ancestry paths with a sum

- depth first search
- Can be subset of path, so need to record the entire ancestry?  what state to store?  Vals as array?
  - Values can be pos or neg, so cannot shortcut there

record ancestry as node state, walk back over state from latest entry to try to sum up our value

corner case
target is 5, path is 3 2 -2 2 ... seems like you have to check entire path sequentially from each termination site back.  A single path might have multiple targets in it

In [ ]:
def num_subsets_match(nancestry: list[int], target: int) -> int:
    sm, count = 0, 0
    for n in nancestry[::-1]:
        sm += n
        if sm == target:
            count+=1
    return count


def path_sum(root: TreeNode, targetSum: int) -> int:
    if root is None:
        return 0
    stack = [(root, [root.val])]
    count = 1 if root.val == targetSum else 0
    while stack:
        node, ancestry = stack.pop()
        if node.left:
            nancestry = ancestry + [node.left.val]
            count += num_subsets_match(nancestry, targetSum)
            stack.append((node.left, nancestry))
        if node.right:
            nancestry = ancestry + [node.right.val]
            count += num_subsets_match(nancestry, targetSum)
            stack.append((node.right, nancestry))

    return count


Given a binary tree root, a node X in the tree is named good if in the path from root to X there are no nodes with a value greater than X.

Return the number of good nodes in the binary tree.


Assess
- root value count as good?  Yes
- empty trees?  Nope
- pos and neg numbers
- equality okay - Yes

Design
depth first search, track max value seen on path.  If value is <= max value, increment count.  Otherwise, add new max value

In [17]:
def _daycare(node: TreeNode, stack: list[tuple], mx: int) -> int:
    # print(node.val, mx)
    if node.val >= mx:
        stack.append((node, node.val))
        return 1
    stack.append((node, mx))
    return 0

def num_good_nodes(root: TreeNode) -> int:
    count = 1 # roots are fundamentally good in nature
    stack = [(root, root.val)]
    while stack:
        node, mx = stack.pop()
        # print(node.val, mx, count)
        if node.left:
            count += _daycare(node.left, stack, mx)
        if node.right:
            count += _daycare(node.right, stack, mx)
    return count


2

## Container With Most Water

You are given an integer array height of length n. There are n vertical lines drawn such that the two endpoints of the ith line are (i, 0) and (i, height[i]).

Find two lines that together with the x-axis form a container, such that the container contains the most water.

Return the maximum amount of water a container can store.

Notice that you may not slant the container.

- Input: height = [1,8,6,2,5,4,8,3,7]
- Output: 49
Explanation: The above vertical lines are represented by array [1,8,6,2,5,4,8,3,7]. In this case, the max area of water (blue section) the container can contain is 49.
Example 2:

Input: height = [1,1]
Output: 1

### Survey
- looking at all possible pairs is niave approach
- Which pairs can I ignore?
  - Step through from 0 to N
    - Can ignore anything to right that is smaller than something seen on left
      - When we see a larger value on right ... can we drop old values?

So storing all previous highs that have been seen ... when can I stop considering the old ones ?


stack state -> (x, y)

In [ ]:
def _area(s: tuple[int,int,int]) -> int:
    return (s[1]-s[0])*s[2]

# reverse order if we see something stupid

def max_size(heights: list[int]) -> int:
    # It helps to draw out scenarios and walk through the stack state
    stack, i, mx = [(0,0,heights[0])], 0, 0
    for h in heights[1:]:
        i += 1
        for j,s in enumerate(stack):
            # Forms a box with lower height than our candidate and a later start index, so the max potential area with 
            # any later candidate would always be less.
            # Check to see if this box is a max, but do not consider it as a candidate start
            if s[2] > h:
                mx = max(_area((s[0],i,h)), mx)
            # else - update end index for any candidates that had lower heights, since they are still intact
            else:
                stack[j] = (s[0], i, s[2])

        # hit a new start candidate (higher potential), track it
        if h > stack[-1][2]:
            stack.append((i,i, h))

    # compare areas of all tracked candidates as well as our intermediate calculations
    return max(mx, max([_area(s) for s in stack]))


test_cases = [
    {"in": ([1,2,3,3,1,1,1,1,1], ), "out": 8},
    {"in": ([1,8,6,2,5,4,8,3,7], ), "out": 49},
    {"in": ([1,1],), "out": 1},
]

run_tests(max_size, test_cases)


In [ ]:
# Performance Tests

# TODO this needs to run in under 10 seconds ... yikes
# pyramid ... currently takes 44 seconds
max_size(list(range(1,10000)) + list(range(10000,1,-1)))


# monotonically increasing
# 8.2 seconds
# max_size(list(range(1,10000)))
# 41 seconds
# max_size(list(range(1,20000)))

# .1 second
# max_size(np.random.randint(1,10000,10000))

## Game of Two Stacks

### Problem Statement
Alexa has two stacks of non-negative integers, A and B. Alexa challenges Nick to play the following game:

- In each move, Nick can remove one integer from the top of either A or B.
- Nick keeps a running sum of the integers he removes from the two stacks.
- Nick is disqualified from the game if, at any point, his running sum becomes greater than some integer *max_sum* given at the beginning of the game.
- Nick's final score is the total number of integers he has removed from the two stacks.

Given A, B, and *max_sum* for N games, find the maximum possible score Nick can achieve.

In [ ]:
def is_new(a, b, visited: set) -> bool:
    "check if visited, mark as visited if not"
    if (a,b) in visited:
        return False
    visited.add((a,b))
    return True

# can probably quit after hitting 20 branches with same result?
def two_stacks_dfs(max_sum, stack_a, stack_b):
    mx = 0
    Q, visited = deque(), set()
    if stack_a:
        Q.append((stack_a[0], 1, 0))
    if stack_b:
        Q.append((stack_b[0], 0, 1))
    while Q:
        rsum, a_idx, b_idx = Q.pop()
        if rsum > max_sum:
            mx = max(a_idx+b_idx-1, mx)
        elif a_idx == len(stack_a) and b_idx == len(stack_b):
            return a_idx+b_idx
        else:
            if a_idx < len(stack_a) and is_new(a_idx+1, b_idx, visited):
                Q.append((rsum + stack_a[a_idx], a_idx+1, b_idx))
            if b_idx < len(stack_b) and is_new(a_idx, b_idx+1, visited):
                Q.append((rsum + stack_b[b_idx], a_idx, b_idx+1))
    return mx

def two_stacks_bfs(max_sum, stack_a, stack_b):
    mx = 0
    Q, visited = deque(), set()
    if stack_a:
        Q.append((stack_a[0], 1, 0))
    if stack_b:
        Q.append((stack_b[0], 0, 1))
    while Q:
        rsum, a_idx, b_idx = Q.popleft()
        if rsum > max_sum:
            mx = max(a_idx+b_idx-1, mx)
        elif a_idx == len(stack_a) and b_idx == len(stack_b):
            return a_idx+b_idx
        else:
            if a_idx < len(stack_a) and is_new(a_idx+1, b_idx, visited):
                Q.append((rsum + stack_a[a_idx], a_idx+1, b_idx))
            if b_idx < len(stack_b) and is_new(a_idx, b_idx+1, visited):
                Q.append((rsum + stack_b[b_idx], a_idx, b_idx+1))
    return mx


inputs =[
    ((10, [4,2,4,6,1], [2,1,8,5]), 4),  # 4
    ((15, [10,1,1,1,1,1], [5,5,5]), 6),  # 6
    ((15, [1,1,1,8,1,1,1], [2,2,2,2,2,2]), 9),  # 9
    ((20, [1,1], [1,1]), 4),  # 4
    ((0, [1], [1]), 0) # 0
]

for i, o in inputs:
    r = two_stacks_dfs(*i)
    if o != r:
        print(i, f"Expected: Found: {o} : {r}")

In [ ]:
# Another test case
games, game = [], None
with open("C:/Users/tom/Downloads/twostacks_input03.txt", "r") as fp:
    for i, line in enumerate(fp.readlines()):
        if i == 0:
            continue
        line_items = [int(k) for k in line.strip().split()]
        if (i-1)%3==0:
            if game:
                games.append(tuple(game))
            game = [line_items[-1]]
        else:
            game.append(line_items)
if game:
    games.append(game)

with open("C:/Users/tom/Downloads/twostacks_output03.txt", "r") as fp:
    answers = [int(s.strip()) for s in fp.readlines()]

for ins, out in zip(games, answers):
    calc = two_stacks_bfs(*ins)
    if calc != out:
        print(i, f"Expected: Found: {out} : {calc}")

In [ ]:
games = []
for i in range(10):
    n,m = np.random.randint(1,int(10e5),2)
    A = np.random.randint(1, int(10e6), n).tolist()
    B = np.random.randint(1, int(10e6), m).tolist()
    mx = np.random.randint(1,int(10e8),1)[0]
    games.append([mx, A, B])

In [ ]:
for g in games:
    two_stacks_bfs(*g)

## Merging Communities

### Problem Statement
People connect with each other in a social network. A connection between Person i and Person j is represented as M(i,j). When two persons belonging to different communities connect, the net effect is to merge the communities to which i and j belong.

At the beginning, there are N people representing N communities (themselves). Suppose person i and j connected and later j and k connected, then i,j, and k will belong to the same community.

There are two types of queries:

Communities containing persons i and j are merged if they belong to different communities.
Print the size of the community to which person  belongs.

Input Format

The first line of input contains 2 space-separated integers  and , the number of people and the number of queries.
The next  lines will contain the queries.

$n < 10^5$<br>
$q < 2*10^5$

In [ ]:
lines = ["3 6", "Q 1", "M 1 2", "Q 2", "M 2 3", "Q 3", "Q 2"]
output = [1,2,3,3]

## Cricket Team Strength (unoptimized)

### Problem Statement
A cricket match is going to be held. The field is represented by a 1D plane. A cricketer, Mr. X has N favorite shots. Each shot has a particular range, $(x_0,x_1)$. Each player on the opposite team can field only in a particular range, $(y_0,y_1)$. For a set of N shots and M players, determine the total strength of the players relative to Mr X.  The strength of a player is the number of shots covered (range of player overlaps range of shot) by that player, and the strength of a team is the sum of all player strengths

$1 \leq N, M \leq 10^5 $ <br>
$1 \leq x,y \leq 10^8 $

In [ ]:
# region  Brute Force

def can_cover(s0, s1, p0, p1) -> bool:
    if s0 <= p1 and s1 >= p0:
        return True


def team_strength_v0(shots, players) -> int:
    strength = 0
    for shot in shots:
        for player in players:
            strength += 1 if can_cover(*shot, *player) else 0
    return strength

#endregion Brute

# region 2
from operator import itemgetter

def team_strength_v1(shots, players) -> int:
    sshots = list(sorted(shots, key=itemgetter(0)))
    pQ = list(sorted(players, key=itemgetter(0)))

    strength = 0
    p0, p1 = pQ[0]
    for s0, s1 in sshots:
        # print(f"Shot {s0,s1}")
        i = 0
        while p0 <= s1:
            if p1 >= s0:
                # print(f"Adding {p0, p1}")
                strength += 1
                i += 1
            else:
                pQ.pop(i)  # Do not consider this one anymore going forward
            if not pQ or i == len(pQ):
                break
            p0, p1 = pQ[i]
        p0, p1 = pQ[0]

    return strength

# endregion

In [ ]:
def gen_inputs(mx_num: int, mx_rng: int) -> tuple:
    num_shots = np.random.randint(1, mx_num, 1)[0]
    num_players = np.random.randint(1, mx_num, 1)[0]

    shots = [(a,b) for a,b in zip(np.random.randint(1, mx_rng/2, num_shots), np.random.randint(mx_rng/2, mx_rng, num_shots))]
    players = [(a,b) for a,b in zip(np.random.randint(1, mx_rng/2, num_players), np.random.randint(mx_rng/2, mx_rng, num_players))]
    return (shots, players)


test_cases = [
    {"in": ([(1, 2), (2, 3), (4, 5), (6, 7)], [(1, 5), (2, 3), (4, 7), (5, 7)]), "out": 9},
    {"in": gen_inputs(int(10e2), int(10e4)), "out": None},
    {"in": gen_inputs(int(10e3), int(10e5)), "out": None},
    # {"in": gen_inputs(int(10e4), int(10e6)), "out": None},
]

In [ ]:
t0 = perf_counter()
team_strength_v0(*test_cases[1]["in"])
t1 = perf_counter()
print(t1-t0)

In [ ]:
t0 = perf_counter()
team_strength_v0(*test_cases[1]["in"])
t1 = perf_counter()
print(t1-t0)

In [ ]:
# This approach is so bad that it makes me question Prima code ... nearly 29 seconds just to build this thing, while brute force completes the entire problem in 7 seconds
def coverage_of(scores: np.array, mx_rng: int) -> np.array:
    coverage = np.zeros(mx_rng+1, dtype=int)

    for x0,x1 in scores:
        coverage[x0] += 1
        coverage[x1+1] -= 1
        # np.add.at(coverage, [p0], 1)

    for i in range(1,len(coverage)):
        coverage[i] = coverage[i] + coverage[i-1]
    return coverage

In [ ]:
shots, players = test_cases[0]["in"]
mx_rng = int(10)
scoverage = coverage_of(shots, mx_rng)
pcoverage = coverage_of(players, mx_rng)

In [ ]:
pcoverage - scoverage

In [ ]:
# approach 1 ... max is slow
strength = 0
for s0,s1 in shots:
    strength += max(pcoverage[s0:s1+1])

strength

# Common Algorithms

## Insertion Sort

In [ ]:
# iterate to the right, keeping the left portion sorted.  Insert into the sorted bit as you go

def _index_of(A: list[int], val: int) -> int:
    for i in range(len(A)):
        if A[i] >= val:  # preserve order?
            return i
    return len(A)

# Value is not in the list
def bsearch(A: list[int], val: int) -> int:
    if len(A) == 1:
        return 0 if val < A[0] else 1
    
    i = int(len(A)/2)
    if val < A[i]:
        return bsearch(A[0:i], val)
    return bsearch(A[i:], val) + i


def insert_sort(A: list) -> None:
    for i in range(1,len(A)):
        v = A[i]
        loc = _index_of(A[:i], v)   # Here, we assume that the 
        # loc = bsearch(A[:i], v)    # Here we assume v is not in the list being searched
        for j in range(i, loc, -1):
            A[j] = A[j-1]
        A[loc] = v


mx = 50
ints = np.random.randint(0,mx,31).tolist()
print(ints)

insert_sort(ints)
print(ints)

## Merge Sort

In [ ]:
def merge_sorted(A, B):
    ai,bi = 0,0
    rval = []
    while ai < len(A) and bi < len(B):
        if A[ai] < B[bi]:
            rval.append(A[ai])
            ai += 1
        else:
            rval.append(B[bi])
            bi +=1
    if bi < len(B):
        rval += B[bi:]
    else:
        rval += A[ai:]
    return rval


def merge_sort(A: list[int]) -> list[int]:
    if len(A) == 2:
        return A if A[0] <= A[1] else A[::-1]
    if len(A) == 1:
        return A
    i = int(len(A)/2)
    return merge_sorted(merge_sort(A[0:i]), merge_sort(A[i:]))


mx = 50
ints = np.random.randint(0,mx,31).tolist()
print(ints)

r = merge_sort(ints)
print(r)

## Quick Sort

In [ ]:
def _swap(A, i,j):
    tmp = A[j]
    A[j] = A[i]
    A[i] = tmp

def _qs_partition(A:list[int], s:int, e: int) -> None:
    j = s
    for i in range(s,e):
        if A[i] < A[e]:
            _swap(A,i,j)
            j += 1
    _swap(A,j,e)
    return j

def _quicksort(A, s,e) -> None:
    if s < e:
        q = _qs_partition(A,s,e)
        _quicksort(A,s,q-1)
        _quicksort(A,q+1,e)

def quicksort(A):
    _quicksort(A, 0, len(A)-1)



mx = 50
ints = np.random.randint(0,mx,31).tolist()
print(ints)

quicksort(ints)  # in place sort
print(ints)


## Count Sort (Constrained Linear)

In [ ]:
# count occurrences of each number, convert that to how many are < each, put elements in correct place
def count_sort(A: list[int], mx: int) -> list[int]:
    cnts = [0 for _ in range(mx+1)]
    for a in A:
        cnts[a] += 1
    for i in range(1,len(cnts)):  # how many are less than or equal to each value
        cnts[i] = cnts[i] + cnts[i-1]
    
    rval = [0 for _ in A]
    for a in A:
        rval[cnts[a]-1] = a
        cnts[a] -= 1  # decrement since next instance of this value goes before
    return rval

mx = 50
ints = np.random.randint(0,mx,31).tolist()
print(ints)

r = count_sort(ints, mx)
print(r)

# Completed Problems

## Huffman decoding

### Problem Statement
Huffman coding assigns variable length codewords to fixed length input characters based on their frequencies. More frequent characters are assigned shorter codewords and less frequent characters are assigned longer codewords. All edges along the path to a character contain a code digit. If they are on the left side of the tree, they will be a 0 (zero). If on the right, they'll be a 1 (one). Only the leaves will contain a letter and its frequency count. All other nodes will contain a null instead of a character, and the count of the frequency of all of it and its descendant characters.

In [ ]:
# Enter your code here. Read input from STDIN. Print output to STDOUT

def decodeHuff(root: Node, encoded: str) -> str:
    node, rval = root, ""
    for e_str in encoded:
        if e_str == "0":
            node = node.left
        else:
            node = node.right
        if node.right is None and node.left is None:
            rval += node.data
            node = root
    print(rval)
    

## Find pairs with sum
Given an unsorted integer array, find all pairs with the given sum in it.

In [ ]:
def brute_force(A: list[int], target: int) -> list[tuple[int, int]]:
    rval = set()
    for i in range(len(A)):
        for j in range(i+1,len(A)):
            if A[i] + A[j] == target:
                rval.add((A[i], A[j]) if A[j] < A[i] else (A[j], A[i]))
    return rval


def sort_method(A:list[int], target: int):
    B = [a for a in A if a <= target]

    B = list(sorted(B))

    i, j = 0, len(B)-1

    rvals = [(0,0)]
    while i < j:
        sm = B[i] + B[j]
        if sm == target:
            cand = (B[i],B[j])
            if rvals[-1] != cand:
                rvals.append((B[i],B[j]))
            i += 1
            j -= 1
        elif sm > target:
            j -= 1
        else:
            i += 1

    return rvals[1:]


test_cases = [
    {"in": ([8, 7, 2, 5, 3, 1], 10), "out": [(8, 2),(7, 3)]},

]

In [ ]:
A = np.random.randint(0,100,5000)
t = 100

t0 = perf_counter()
m1 = brute_force(A, t)
t1 = perf_counter()
print(f"Using v0, len is {len(m1)} , time is: {t1 - t0}")

t0 = perf_counter()
m2 = sort_method(A, t)
t1 = perf_counter()
print(f"Using v1, len is {len(m2)} , time is: {t1 - t0}")

## Queue from 2 stacks

### Problem Statement
In this challenge, you must first implement a queue using two stacks and process queries of type:

1. x: Enqueue element  into the end of the queue.
2. Dequeue the element at the front of the queue.
3. Print the element at the front of the queue.

Up to 10e5 queries can be provided

In [ ]:
# expected to print 14 \n 14 
inp = [(1,42),(2,),(1,14),(3,),(1,28),(3,),(1,60),(1,78),(2,),(2,)]

# enqueue -> append to stack_1
# deque -> if stack_2 is empty, pop everything from stack_1 onto stack_2.  Now pop from stack 2
# Important to note that stack_1 can be ignored on a dequeue call, since the first len(stack_2) items in the queue are already on stack 2
# As a result, we can do an easy pop from stack 2 until it is empty, then we just pop everything from stack 1

def _swap_if_needed(dq: list, eq: list):
    if not dq:
        while eq:
            dq.append(eq.pop())

eq_stack, dq_stack = [], []
for args in inp:
    if args[0] == 1:
        x = args[1]
        eq_stack.append(x)
    elif args[0] == 2:
        # key is that dq_stack is kind of like a buffer that only
        # needs to be refilled once empty,
        _swap_if_needed(dq_stack, eq_stack)
        if dq_stack:
            dq_stack.pop()
    else:
        _swap_if_needed(dq_stack, eq_stack)
        print(dq_stack[-1])

## Truck Tour

### Problem Statement
Suppose there is a circle. There are *N* petrol pumps on that circle. Petrol pumps are numbered 0 --> (N-1). You have two pieces of information corresponding to each of the petrol pumps:
1. the amount of petrol, *p*, that particular petrol pump will give
2. the distance, *d*, from that petrol pump to the next petrol pump.

Initially, you have a tank of infinite capacity carrying no petrol. You can start the tour at any of the petrol pumps. Calculate the first point from where the truck will be able to complete the circle. Consider that the truck will stop at each of the petrol pumps. The truck will move one kilometer for each litre of the petrol.

**Input**<br>
list of tuples corresponding to petrol pumps

**Output**<br>
An integer which will be the smallest index of the petrol pump from which we can start the tour. (zero indexed)

**Constraints**<br>
$1 < N < 10^5$<br>
$ 1 \leq p,d < 10^9$

In [ ]:
# brutish approach
# start at lowest pump i
# if you can get to the end, return i.  Otherwise, try with i+1


# faster approach ... binary search?

# note that it is a circle ... so 0 -> 1 -> ... -> n-2 -> n-1 -> 0 -> ...
def can_complete(pumps: list[tuple], start_idx: int) -> bool:
    circle_indices = list(range(start_idx, len(pumps))) + list(range(0, start_idx))
    tank_surplus = 0
    for i in circle_indices:
        tank_surplus += (pumps[i][0]-pumps[i][1])
        if tank_surplus < 0:
            return False
    return True

# if I can't make it from i, can I make it from i-1?  Maybe, if p>d at i - 1
# if I can't make if from i, can I make it from i+1?
# No, it doesn't have the properties needed for a binary search

# Bronze Age
def earliest_start_v0(pumps: list[tuple]) -> int:
    for i in range(len(pumps)):
        if can_complete(pumps, i):
            return i



# start at 0, 

# what is the state?  index (0), surplus
# if we hit a pump without a surplus, it is not a contestant
# walk through tracking surplus at each point
# (index, tank_surplus)
# 

# 100000000, 1  1,10 1,10 1,10 ... 1, 10

# Imperial age solution
def _surplus(p,d):
    return p - d

def earliest_start_v1(pumps: list[tuple]) -> int:
    k, tank_surplus = 0,0
    for i in range(len(pumps)):
        tank_surplus += _surplus(*pumps[i])
        if tank_surplus < 0:
            k = i + 1
            tank_surplus = 0
    return k




ins = [(1,5),(10,3),(3,4)]  # out = 1

earliest_start_v1(ins)

## Merge k sorted arrays
You are given an array of k lists, each sorted in ascending order. Merge all of them into a sorted list and return it.  (19 minutes)

In [ ]:
def merge_sorted(A: list[int], B: list[int]) -> list[int]:
    a,b = 0,0
    rval = []
    while a < len(A) and b < len(B):
        if A[a] < B[b]:
            rval.append(A[a])
            a += 1
        else:
            rval.append(B[b])
            b += 1
    if a < len(A):
        rval = rval + A[a:]
    if b < len(B):
        rval = rval + B[b:]
    return rval

In [ ]:
C = list(sorted(np.random.randint(0,10000,10000).tolist()))
D = list(sorted(np.random.randint(0,10000,10000).tolist()))

E = merge_sorted(C,D)


## Book reading

### Problem Statement
You are given a book with chapters 0 -> N, where chapter `i` has $pages[i]$ pages.  When reading, you can read *pages_per_day* OR until you reach the end of the current chapter.  If you have X days to finish a book, what is the minimum number of pages per day that you can read to finish the book in time?

In [ ]:
def num_days_to_complete(pages: list[int], pages_per_day: int, days_to_read: int) -> int:
    pages_left, page_idx, day_count = pages[0], 0, 0
    while page_idx < len(pages)-1:
        pages_left = pages_left-pages_per_day
        day_count += 1
        if day_count > days_to_read:
                return -1
        if pages_left <= 0:
            page_idx += 1
            pages_left = pages[page_idx]
    
    while pages_left > 0:
        pages_left = pages_left-pages_per_day
        day_count += 1

    if day_count > days_to_read:
        return -1
    return day_count


def bsearch(pages, candidates, num_days) -> int:
    if len(candidates) == 1:
        r = num_days_to_complete(pages, candidates[0], num_days)
        return candidates[0] if r > -1 else -1
    if len(candidates) == 2:
        r = bsearch(pages, candidates[0:1], num_days)
        if r > -1:
            return r
        return bsearch(pages, candidates[1:], num_days)
    i = int(len(candidates)/2)
    r = num_days_to_complete(pages, candidates[i], num_days)
    if r == -1:
        return bsearch(pages, candidates[i+1:], num_days)
    else:
        return bsearch(pages, candidates[0:i], num_days)



def min_pages_per_day(pages: list[int], num_days: int) -> int:
    "return -1 if it is impossible"
    # if you read fewer than the average, you can't make it
    lower_bound = int(sum(pages) / num_days) - 1
    upper_bound = max(pages)
    if len(pages) > num_days:
        return -1
    if len(pages) == num_days:
        return max(pages)
    
    candidates = list(range(lower_bound, upper_bound))
    return bsearch(pages, candidates, num_days)


In [ ]:
simple_cases = [
    # pages, num_days, answer
    ((3, 4, 5, 3, 5), 5, 5),
    ((1,1,1,1,1,1,1,1,1,1), 5, -1),
    ((1,5000,1,1), 5, 2500)
]

performance_cases = []

num_tests = 10
for _ in range(num_tests):
    num_days = np.random.randint(1,1e5,1)[0]
    num_chapters = np.random.randint(1, 1e5, 1)[0]
    pages = np.random.randint(0,1e7,num_chapters).tolist()
    performance_cases.append((pages, num_days))

In [ ]:
for pages, num_days, answer in simple_cases:
    result = min_pages_per_day(pages, num_days=num_days)
    print(answer == result or (answer, result))

print("==================")
for pages, num_days in performance_cases:
    t0 = perf_counter()
    r = min_pages_per_day(pages, num_days=num_days)
    t1 = perf_counter()

    print(f"# chapters: {len(pages)} in {num_days} days, avg chapter size is {int(sum(pages)/len(pages))}")
    print(f"Minimum reading speed is: {'NA' if r < 0 else r} pages per day")
    print(f"Time: {t1-t0} seconds")
    print("==================")

## Reverse Linked List

### Problem Statement
Given the head of a linked list, reverse the nodes of the list `k` at a time, and return the modified list.
`k` is a positive integer and is less than or equal to the length of the linked list. If the number of nodes is not a multiple of `k` then left-out nodes, in the end, should remain as it is.
You may not alter the values in the list's nodes, only nodes themselves may be changed.

In [ ]:
def linklist_str(head: Node) -> str:
	s = []
	c = head
	while c.right:
		s.append(str(c.data))
		c = c.right
	s.append(str(c.data))
	return ", ".join(s)


class LinkedList:
	def __init__(self, size: int):
		data = np.random.randint(0,500,size)
		head = Node(data=data[0])
		prev = head
		for i in data[1:]:
			curr = Node(data=i, prev=prev)
			prev.right = curr
			prev = curr
		self.head = head
		self.size = size

	def __sizeof__(self) -> int:
		return self.size
	
	def __str__(self) -> str:
		if self.head is None:
			return ""
		return linklist_str(self.head)


In [ ]:
llist = LinkedList(20)

print(llist)

def reverse_list(head: Node) -> Node:
    ref = head
    while ref.right:
        x = ref.right
        ref.right = ref.left
        ref.left = x
        ref = x
    ref.right = ref.left
    ref.right = None
    return ref


new_head = reverse_list(llist.head)
linklist_str(new_head)

### Singely Linked List

In [ ]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

def reverse_list(head: ListNode) -> ListNode:
    if head is None:
        return head
    node = head
    prev = None
    while node.next:
        tmp = node.next   # store reference
        node.next = prev  # override stored reference
        prev = node       # update previous node reference
        node = tmp        # move to the next node in the list
    node.next = prev      # add the last link

    return node

## Largest rectangle

### Problem Statement
Skyline Real Estate Developers is planning to demolish a number of old, unoccupied buildings and construct a shopping mall in their place. Your task is to find the largest solid area in which the mall can be constructed.  There are a number of buildings in a certain two-dimensional landscape. Each building has a height, given by $h$. If you join adjacent buildings, they will form a solid rectangle of area   (len * min_height)

In [ ]:
def largestRectangle(heights):
    max_area, stack = 0, []
    for i, height in enumerate(heights):
        idx = i  # track the starting index for the block with min_height = height
        
        # check our history for streaks with a higher height.  Those streaks are at an
        # end.  Remove the tracker for them, calculate the corresponding area, and save 
        # it if it represents the max_area seen so far
        while stack and height < stack[-1][1]:
            prev_idx, h = stack.pop(-1)
            max_area = max(max_area, h*(i-prev_idx))
            idx = prev_idx
            
        # enter our new streak with min height of height
        stack.append((idx, height))
        
    # Everything left on the stack represents a streak that goes from
    # the stored start index to the end of the array.  Calculate areas for these blocks
    for i, height in stack:
        max_area = max(max_area, height*(len(heights)-i))
            
    return max_area

# region v2
def largest_area(heights: list[int]) -> int:
    stack, max_area = [], 0
    for i, height in enumerate(heights):
        append_idx = i
        while stack and stack[-1][1] > height:
            prev_idx, prev_height = stack.pop()
            max_area = max(max_area, prev_height*(i-prev_idx))
            append_idx = prev_idx
        stack.append((append_idx, height))
    # only items left go through to the last building
    for prev_idx, prev_height in stack:
        max_area = max(max_area, prev_height*(len(heights)-prev_idx))
    return max_area
#endregion

test_cases = [
    {"in": (1,2,3,4,5), "out": 9},
    {"in": (1, 3, 5, 9, 11), "out": 18},
    {"in": (11, 11, 10, 10, 10), "out": 50},

]

## Plate stack

### Problem Statement

You are a waiter at a party. There is a pile of numbered plates, `numbered_plates`. Create an empty `answers` array. At each iteration, `i` , remove each plate from the top of the `A` stack in order. Determine if the number **on** the plate is evenly divisible by the $i^{th}$ prime number. If it is, stack it in `B` pile . Otherwise, stack it in $A_i$ stack . Store the values in `B`  from top to bottom in `answers`. In the next iteration, do the same with the values in stack `A`. Once the required number of iterations is complete, store the remaining values from `A` in `answers`, again from top to bottom. Return the `answers`

Example

num_iterations = `2`
numbered_plates = `[2,3,4,5,6,7]`

An abbreviated list of primes is `[2,3,5,7,11]`.

Start:
Add the plates from `numbered_plates` to the `A` stack.

Begin iterations. On the first iteration, check if items are divisible by `2` (the first prime).
$A = [7,5,3]$
$B = [6,4,2]$
Elements are taken off of B and appended to the answers **array**
$A = [7,5,3]$
$B = []$
$answers = [2,4,6]$

On the second iteration, test if $A$ elements are divisible by `3` (the second prime).
$A = [7,5]$
$B = [3]$
$answers = [2,4,6]$

Move `B` elements to `answers`.
$A = [7,5]$
$B = []$
$answers = [2,4,6,3]$

All iterations are complete, so move the remaining elements in `A` to `answers`.
$A = []$
$B = []$
$answers = [2,4,6,3,5,7]$

Return `answers`

In [ ]:
BASE_PRIMES = [2,3,5,7,11,13,17,19,23,29, 31, 37]

# every non prime number is divisible by prime numbers
def gen_primes(num_primes: int):
    primes, c = BASE_PRIMES, BASE_PRIMES[-1]
    while len(primes) < num_primes+2:
        c += 1
        is_prime = True
        for p in primes:
            if c % p == 0:
                is_prime = False
                break
            if p > (c**0.5):
                break
        if is_prime:
            primes.append(c)
    return primes
    

def waiter(numbers, q):
    primes = gen_primes(q-1)
    A, B, answers = [n for n in numbers], [], []
    for i in range(q):
        pi = primes[i]
        A_next = []
        for a in A[::-1]:
            if a % pi == 0:
                B.append(a)
            else:
                A_next.append(a)
        A = A_next

        for _ in range(len(B)):
            answers.append(B.pop(-1))
        
    for _ in range(len(A)):
        answers.append(A.pop(-1))
    return answers

In [ ]:
def _right(nums: list[int], i: int) -> int:
    for j in range(i+1, len(nums)):
        if nums[j] > nums[i]:
            return j+1
    return 0


def _left(nums: list[int], i: int) -> int:
    for j in range(i-1, -1, -1):
        if nums[j] > nums[i]:
            return j+1
    return 0
    

def _left_fast(stack: list[int], elem: int, idx: int) -> int:
    while stack and stack[-1][1] <= elem:
        stack.pop(-1)
      
    rval = -1
    if stack:
        rval = stack[-1][0]  # get index
    stack.append((idx, elem))
    return rval+1
    

def _right_fast(nums: list[int], progressives: list[tuple[int, int]], i: int) -> int:
    while progressives and i >= progressives[0][0]:
        progressives.popleft()
    
    if not progressives:
        if i < len(nums):
            for j in range(i+1, len(nums)):
                if nums[j] > nums[i]:
                    return j+1
        return 0
    
    if nums[i] < progressives[0][1]:
        for j in range(i+1, progressives[0][0]+1):
            if nums[j] > nums[i]:
                return j+1
        # search between here and that index
    else:
        for j, n in progressives:
            if n > nums[i]:
                return j+1
    return 0

def _progressive_larger(arr, i, j):
    if i >= j:
        return []
    r = [(i, arr[i])]
    for k in range(i+1, j):
        if arr[k] > r[-1][1]:
            r.append((k, arr[k]))
    return deque(r)


def solve_v0(arr: list[int]):
    mx = 0
    for i in range(len(arr)):
        lft = _left(arr, i)
        rgt = _right(arr, i)
        mx = max(mx, lft*rgt)
    return mx


def solve_v1(arr: list[int]):
    mx = 0
    lstack = []
    for i, elem in enumerate(arr):
        lft = _left_fast(lstack, elem, i)
        rgt = _right(arr, i)
        mx = max(mx, lft*rgt)
    return mx


def solve_v2(arr: list[int]):
    mx = 0
    lstack = []
    right = _progressive_larger(arr, 1, len(arr))
    for i, elem in enumerate(arr):
        lft = _left_fast(lstack, elem, i)
        # print(elem, right)
        rgt = _right_fast(arr, right, i)
        # print(rgt)
        mx = max(mx, lft*rgt)
    return mx


# I am not sure why this is correct?  How is it optimal that the next value ... I think that this is wrong and fails with
# array = [4,5,6,2,3,10,9]
def optimal(array):
    returned_values_left: list[int] = []
    returned_values_right: list[int] = []
    for index, item in enumerate(array):
        if index == 0:
            returned_values_left.append(0)
        else:
            if item < array[index - 1]:
                returned_values_left.append(index)
            else:
                returned_values_left.append(0)

        if index == len(array)-1:
            returned_values_right.append(0)
        else:
            if item < array[index + 1]:
                returned_values_right.append(index + 2)
            else:
                returned_values_right.append(0)
    return [returned_values_left, returned_values_right]

import sys
def solve_other(arr):
    value = sys.maxsize * -1
    [left, right] = optimal(arr)

    for position in range(len(arr)):
        new_value = left[position] * right[position]
        if new_value > value:
            value = new_value

    return value

In [ ]:
solve_v1([4,5,6,2,3,10,9])

In [ ]:
arr = np.random.randint(0, 1e6, 999999)

t0 = perf_counter()
solve_v0(arr)
t1 = perf_counter()
print(f"Using v0: {t1 - t0}")

t0 = perf_counter()
solve_v1(arr)
t1 = perf_counter()
print(f"Using v1: {t1 - t0}")


t0 = perf_counter()
solve_v2(arr)
t1 = perf_counter()
print(f"Using v2: {t1 - t0}")


t0 = perf_counter()
solve_other(arr)
t1 = perf_counter()
print(f"Using others solution: {t1 - t0}")

## Build a binary search tree
Build a binary search tree from a list of random integers and print its contents using an inorder traversal method

In [ ]:
class BSTree:
	def __init__(self, root_data: int):
		self.root = Node(data=root_data)

	def add_node(self, at_node: Node, data: int):
		if at_node.data > data:
			if at_node.left is None:
				new_node = Node(data=data, parent=at_node)
				at_node.left = new_node
			else:
				self.add_node(at_node.left, data)
		else:
			if at_node.right is None:
				new_node = Node(data=data, parent=at_node)
				at_node.right = new_node
			else:
				self.add_node(at_node.right, data)

	@staticmethod
	def _print_node(node: Node):
		if node and node.data is not None:
			print(node.data)

	@staticmethod
	def _inorder_print(node: Node):
		if node is not None:
			BSTree._inorder_print(node.left)
			BSTree._print_node(node)
			BSTree._inorder_print(node.right)

	def pprint(self, mode: str="inorder"):
		if mode == "inorder":
			BSTree._inorder_print(self.root)
		else:
			raise NotImplementedError("Only 'inorder' printing is supported at this time.")


# if __name__ == "__main__":

def build_BSTree(num_items: int=20) -> BSTree:
	ints = np.random.randint(0,100,num_items)
	np.random.shuffle(ints)  # shuffle since sorted arrays are common and are worse case for BSTree

	print(ints)
	tree = BSTree(ints[0])
	for d in ints[1:]:
		tree.add_node(tree.root, d)
	tree.pprint("inorder")
	return tree

## Tree max path sum
Given the root of a binary tree, return the maximum path sum of any non-empty path.

In [ ]:
# This assumes that you have executed the BSTree code cell above to define these classes

def leaf_ancestries(node: Node, ancestry: list[Node], results: list[list[Node]]) -> None:
	if node is None:
		return
	if node.left is None and node.right is None:
		results.append(ancestry)
		return
	if node.left is not None:
		leaf_ancestries(node.left, ancestry + [node.left], results)
	if node.right is not None:
		leaf_ancestries(node.right, ancestry + [node.right], results)


def main():
	tree = build_BSTree(20)
	ancestries = []
	leaf_ancestries(tree.root, [tree.root], ancestries)
		
	max_sum = 0
	for a in ancestries:
		print([z.data for z in a])
		s = sum([z.data for z in a])
		if s > max_sum:
			max_sum = s
	print(f"Max path sum is {max_sum}")

## Reverse Nodes

### Problem Statement
We define depth of a node as follows:
- The root node is at depth 1.
- If the depth of the parent node is d, then the depth of current node will be d+1.
- 
Given a tree and an integer, `k`, in one operation, we need to swap the subtrees of all the nodes at each depth `h`, where `h` ∈ [`k`, 2`k`, 3`k`,...]. In other words, if `h` is a multiple of `k`, swap the left and right subtrees of that level.

You are given a tree of n nodes where nodes are indexed from 1 to `n` and it is rooted at 1. You have to perform t swap operations on it, and after each swap operation print the in-order traversal of the current state of the tree.

In [ ]:
def assign_depth(node, depth):
    if node.left:
        assign_depth(node.left, depth+1)
    node.depth = depth
    if node.right:
        assign_depth(node.right, depth+1)
        
def build_tree(indices: list[tuple[int, int]]):
    nodes = [Node(data=i+1) for i in range(len(indices))]
    for i, (l, r) in enumerate(indices):
        if l > 0:
            nodes[i].left = nodes[l-1]
        if r > 0:
            nodes[i].right = nodes[r-1]
    assign_depth(nodes[0], 1)
    return nodes[0]


def iorder_traverse(node, rval):
    if node.left:
        iorder_traverse(node.left, rval)
    rval.append(node.data)
    if node.right:
        iorder_traverse(node.right, rval)


# swapping at depth doesn't change depth, so we could construct nodes of each depth
def _swap_kids(root, nodes):
    for n in nodes:
        tmp = n.left
        n.left = n.right
        n.right = tmp


def swap_n_return(root, nodes_by_depth: dict[int, list[Node]], max_depth: int, k: int):
    depths = []
    for j in range(1, 1025):
        q = j*k
        if q > max_depth:
            break
        depths.append(q)
    
    for d in depths:
        _swap_kids(root, nodes_by_depth[d])
        
    rval = []
    iorder_traverse(root, rval)
    return rval


def build_depth_list(node, rval):
    if node.left:
        build_depth_list(node.left, rval)
    d = node.depth
    rval[d] = rval.get(d, []) + [node]
    if node.right:
        build_depth_list(node.right, rval)
    

def build_by_depth(root):
    by_depth = {}
    build_depth_list(root, by_depth)
    return by_depth
    

def swap_nodes(indexes, queries):
    root = build_tree(indexes)
    nodes_by_depth = build_by_depth(root)
    max_depth = max(nodes_by_depth.keys())
    return [swap_n_return(root, nodes_by_depth, max_depth, q) for q in queries]


In [ ]:
# queries = [1, 1]
# indices = [(2,3), (-1, -1), (-1, -1)]
# output = ["3 1 2", "2 1 3"]

# queries = [2]
# indices = [(2,3), (-1,4), (-1, 5), (-1, -1), (-1,-1)]
# output = ["4 2 1 5 3"]

queries = [2, 4]
indices = [(2,3), (4, -1), (5, -1), (6, -1), (7,8), (-1, 9), (-1, -1), (10, 11), (-1, -1), (-1, -1), (-1, -1)]
output = ["2 9 6 4 1 3 7 5 11 8 10", "2 6 9 4 1 3 7 5 10 8 11"]


results = swap_nodes(indices, queries)
[" ".join(map(str, o)) for o in results] == output

## Down to Zero

### Problem Statement
You are given Q queries. Each query consists of a single number N. You can perform any of the 2 operations on N in each move:

1. Take $max(a,b)$, where $N = a*b$ and $a > 1$ and $b > 1$
2. Decrease the value of N by 1.

Determine the minimum number of moves required to reduce the value of N to 1.

In [ ]:
def maxes_of_multiples(n: int):
    maxes = []
    for i in range(2, int(math.sqrt(n))+1):
        if n % i == 0:
            maxes.append(max(i, n / i))
    return list(sorted(maxes, reverse=True))


def bfs(Q: deque, visited):
    # print(Q)
    newQ = deque()
    while Q:
        n, d = Q.popleft()
        if n == 1:
            return d+1
        if n not in visited:
            newQ.append((n-1, d+1))
            for mx in maxes_of_multiples(n):
                newQ.append((mx, d+1))
            visited.add(n)
    
    return bfs(newQ, visited)


def down_to_zero(n):
    if n <=3:
        return n
    Q = deque()
    Q.append((n, 0))
    visited = set()
    return bfs(Q, visited)


## Components in a graph

### Problem Statement
There are *2N* nodes in an undirected graph, and a number of edges connecting some nodes. In each edge, the first value will be between 1 and *N*, inclusive. The second node will be between $N+1$ and $2N$, inclusive. Given a list of edges, determine the size of the smallest and largest connected components that have 2 or more nodes. A node can have any number of connections. The highest node value will always be connected to at least 1 other node and single nodes should not be considered in the answer.

[hacker rank problem](https://www.hackerrank.com/challenges/components-in-graph/problem?isFullScreen=true)

In [ ]:
# Basically run a breadth first search on a pair, tracking the nodes that were connected to that pair.
# For pairs that are not connected with any subgraph seen so far, determine its set of nodes
# and add that to the list of all nodes seen so far.

def nodes_in_subgraph(node, connection_map: dict[int, list[int]]) -> set[int]:
    nodes = set()
    Q = deque()
    Q.append(node)
    
    while Q:
        node = Q.popleft()
        connections = connection_map.get(node, [])
        for c in connections:
            if c not in nodes:
                Q.append(c)
        nodes.add(node)
    return nodes


def components_in_graph(edges: list[tuple[int, int]]) -> tuple[int, int]:
    # build a dict to efficiently find all nodes connected to another
    node_connections = {}
    for n, c in edges:
        node_connections[n] = node_connections.get(n, []) + [c]
        node_connections[c] = node_connections.get(c, []) + [n]
    
    visited, lengths = set(), []
    for a, b in edges:
        if a in visited or b in visited:
            continue
        
        nodes = nodes_in_subgraph(a, node_connections)
        visited = visited.union(nodes)
        lengths.append(len(nodes))
    
    lengths = [l for l in lengths if l > 1]
    return [min(lengths), max(lengths)]

In [ ]:
edges = [[1, 6],[2, 7], [3, 8], [4,9], [2, 6]]

components_in_graph(edges)

## Castle on the Grid

### Problem Statement
https://www.hackerrank.com/challenges/castle-on-the-grid/problem?isFullScreen=true

You are given a square grid with some cells open (.) and some blocked (X). Your playing piece can move along any row or column until it reaches the edge of the grid or a blocked cell. Given a grid, a start and a goal, determine the minmum number of moves to get to the goal.

In [ ]:
def move_dir(grid, x, y, n, visited, Q):
    if grid[x][y] == "X":
        return True
    if (x, y) not in visited:
        Q.append((x, y, n+1))
        visited.add((x, y))


def minimum_noves(grid, start: tuple[int,int], goal: tuple[int,int]):
    Q = deque()
    Q.append((start[0], start[1], 0))
    visited = set((start[0], start[1]))
    while Q:
        x, y, num_moves = Q.popleft()
        if x == goal[0] and y == goal[1]:
            return num_moves
            
        for new_x in range(x-1, -1, -1):  # check moving left
            if move_dir(grid, new_x, y, num_moves, visited, Q):
                break
                
        for new_x in range(x+1, len(grid)):  # moving right
            if move_dir(grid, new_x, y, num_moves, visited, Q):
                break

        for new_y in range(y-1, -1, -1):  # moving down
            if move_dir(grid, x, new_y, num_moves, visited, Q):
                break
                
        for new_y in range(y+1, len(grid)):  # moving up
            if move_dir(grid, x, new_y, num_moves, visited, Q):
                break


inputs = [
    {"grid": ["...", ".X.", "..."], "start": (0,0), "goal": (1,2)},
    {"grid": ['.X.','.X.', '...'], "start": (0,0), "goal": (0,2)},
    {"grid": ["...", ".X.", ".X."], "start": (2,0), "goal": (0,2)},
    {"grid": ["...", ".X.", ".X."], "start": (2,0), "goal": (2,2)},
    ]
outputs = [2, 3, 2, 3]


for input, out in zip(inputs, outputs):
    assert minimum_noves(**input) == out

## Poisonous Plants

### Problem Statement
There are a number of plants in a garden. Each of the plants has been treated with some amount of pesticide. After each day, if any plant has more pesticide than the plant on its left, it dies.

You are given the initial values of the pesticide in each of the plants. Determine the number of days after which no plant dies, i.e. the time after which there is no plant with more pesticide content than the plant to its left.

https://www.hackerrank.com/challenges/poisonous-plants

In [ ]:
def poisonous_plants_v1(p: list[int]) -> int:
    cnt = 0
    while p:
        to_remove = []
        for i in range(1, len(p)):
            if p[i] > p[i-1]:
                to_remove.append(i)
        if to_remove:
            for i in to_remove[::-1]:
                p.pop(i)
            cnt += 1
        else:
            return cnt
    

def poisonous_plants_v2(pesticides: list[int]) -> int:
    global_mx, stack = 0, [(pesticides[0], 0)]
    for p in pesticides[1:]:
        local_mx = 0
        while stack and stack[-1][0] >= p:
            _, od = stack.pop()
            local_mx = max(od, local_mx)
        d = local_mx + (1 if stack else 0)  # don't increment time_count if a new low
        global_mx = max(local_mx, global_mx, d)
        stack.append((p, d))
    return max(global_mx, local_mx)


test_cases = [
    ([6, 5, 8, 4, 7, 10, 9], 2),
    ([4, 3, 7, 5, 6, 4, 2], 3),
    ([3, 2, 5, 4], 2),
    ([2, 9,8,7,7,8,3,1], 5),
    ([2, 9,8,7,7,8,8,9,10,8,3,1], 5),
    ([2,9,8,7,7,8,8,9,10,8,8,8,3,1], 6),
    ([2,9,8,7,7,8,8,9,10,8,8,8,6,3,1], 7),
    ([5,9,8,7,7,8,8,9,10,8,8,8,6,3,1], 6),
    ([3,2,1], 0),
    ([1,2,3,5,6], 1)
]

for inp, exp in test_cases:
    s = f"IN: {inp}"
    result = poisonous_plants_v2(inp)
    if exp != result:
        s = s + f" : Exp: {exp} , Calc: {result}"
        print(s)

## Skyscrapers

### Problem Statement
Jim has invented a new flying object called HZ42. HZ42 is like a broom and can only fly horizontally independent of the environment. One day, Jim started his flight from Dubai's highest skyscraper, traveled some distance and landed on another skyscraper of same height! So much fun! But unfortunately, new skyscrapers have been built recently.

Let us describe the problem in one dimensional space. We have in total N skyscrapers aligned from left to right. The $ith$ Skyscraper has a height of $heights[i]$. A flying route can be described as $(i,j)$ and is valid if all skyscrapers between i and j have height <= height[i] and height[i] = height[j].  Note that Jim cannot reach skyscraper N  directly from skyscraper 1 by going 'left'

How many valid routes are there?

$N < 3*10^5$ <br>
$1 <= H[i] < 10^6$ 

In [ ]:
def solve_v0(heights: list[int]) -> int:
    num_routes = 0
    for i, h in enumerate(heights):
        for j in range(i+1, len(heights)):
            if heights[j] > h:
                break
            if heights[j] == h:
                num_routes += 1
    return num_routes*2


def solve_v1(heights: list[int]) -> int:
    stack, cnt = [], 0
    for h in heights:
        while stack and stack[-1] < h:
            stack.pop()
        for stack_h in stack[::-1]:
            if stack_h == h:
                cnt += 1
            else:
                break
        stack.append(h)
    return cnt*2

# same as v1, but instead of storing all instances of value = h in the queue,
# store the number of instances in the current streak alongside the height
def solve_v2(heights: list[int]) -> int:
    stack, cnt = [], 0
    for h in heights:
        streak = 0
        while stack and stack[-1][0] < h:
            stack.pop()
        if stack and stack[-1][0] == h:
            _, streak = stack.pop()
            cnt += streak
        stack.append((h, streak+1))
    return cnt*2


from time import perf_counter
def gen_inputs(max_N: int, max_height: int):
    N = np.random.randint(1, max_N, 1)[0]
    heights = np.random.randint(1, max_height, N)
    return heights


# basic idea here is similiar to the largest rectangle problem, where you will walk over the heights
# and keep track of your history in a stack.  When you reach a building of height h:
# Any ongoing routes that start at a lower building are now ended
# Any ongoing routes that start at a higher building continue
# Any ongoing routes that start at a building of height h could end here, so increment total count accordingly
# Start a new route with height h (add to stack), or extend streak associated with your route 

In [ ]:
solve_v2((3, 2, 1, 2, 3, 3))

In [ ]:
heights = gen_inputs(int(3*10e5), int(10e6))
solve_v0(heights)

In [ ]:
results = []
t0 = perf_counter()
a = solve_v0(heights)
t1 = perf_counter()
results.append((a, t1-t0))
a = solve_v1(heights)
t2 = perf_counter()
results.append((a, t2-t1))
a = solve_v2(heights)
t3 = perf_counter()
results.append((a, t3-t2))

results

In [ ]:
solve_v1(heights)

In [ ]:
test_cases = [
    {"in": ((3, 2, 1, 2, 3, 3),), "out": 8},
    {"in": ((1, 3, 1),), "out": 0},
]

run_tests(solve_v1, test_cases)

## Minimal sum subsequence

### Problem Statement
A subsequence is a sequential subset of an array with the following properties:
1. subset length is 3
2. s[i] < s[j] > s[k]
3. i < j < k

So for array [1,2,5,7,8,5], subsequences are:
1,7,5 and 1,8,5 and 2,7,5 and 2,8,5 and 5,7,5 and 5,8,5

Write a function that takes an array of numbers and outputs the sum for the subsequence with the smallest sum (or -1 if no subsequences exist)

In [ ]:

MAX = 9e11

def get_minimum_sum_v0(arr):
    rval = MAX
    for i, c0 in enumerate(arr[:-2]):
        for j, c1 in enumerate(arr[i+1:]):
            if c0 < c1:
                for c2 in arr[j+i+2:]:
                    if c1 > c2:
                        rval = min(rval, c1+c2+c0)
                        
    return rval if rval < 9e11 else -1


def get_minimum_sum_v1(arr: list[int]) -> int:
    c1 = arr[0]  # at any point, the lowest value ever seen is optimal as triplets first element.  Unconditionally true
    stems, global_min = [], MAX
    for a in arr[1:]:
        # print(a, c1, stems, global_min)
        # check if valid stem
        if a > c1:
            stems.append((c1,a))
        else:
            c1 = a  # lowest value seen so far

        # try to complete all stems
        completed_stems = [s for s in stems if a < s[1]]
        min_stem, min_val = None, MAX
        for stem in completed_stems:
            v = stem[0] + stem[1] + a
            if v < min_val:
                min_stem = stem
                min_val = v
        
        if min_stem:
            completed_stems.remove(min_stem)
        global_min = min(min_val, global_min)

        # remove all completed_stems that are not min_stem from stems
        if completed_stems:
            stems = [s for s in stems if s not in completed_stems]
        
    return -1 if global_min == MAX else global_min

        


test_cases = [
    {"in": ([1,2,5,7,8,5],), "out": 13},
    {"in": ([1,2,3,4,5,6,7],), "out": -1},
]

run_tests(get_minimum_sum_v0, test_cases)
run_tests(get_minimum_sum_v1, test_cases)

In [ ]:
# Performance test

A = np.random.randint(1, int(10e6), 1000).tolist()

t0 = perf_counter()
a = get_minimum_sum_v0(A)
t1 = perf_counter()
b = get_minimum_sum_v1(A)
t2 = perf_counter()

a, (t1-t0), b, (t2-t1)

## Plant Potting

### Problem Statement
You have a long flowerbed in which some of the plots are planted, and some are not. However, flowers cannot be planted in adjacent plots.

Given an integer array flowerbed containing 0's and 1's, where 0 means empty and 1 means not empty, and an integer n, return true if n new flowers can be planted in the flower bed without violating the no-adjacent-flowers rule and false otherwise.

### survey
- Adjacent plots CANNOT have flowers
- Can N new plants be planted
  - Need to optimize how to plant empty spaces?
  - X X X is empty -> would you ever plant at index 1?  Yes, if either side was full
  - So maybe try to plan in each empty slot

### Plan
- One needs sequences of at least 3 empty slots for each plant
  - seq of 3,4 empty slots -> 1 new plant
  - seq of 5,6 empty slots -> 2 new plants   Y _ _ _ _ _ Y ->  Y_ X_ X_Y
  - seq of 7,8 -> 3
  - num new plants = floor(seq_len-1 / 2)

- Corner case of the start and end, where only 2 empty slots are needed
  - make logic easier by appending a 0 to both sides

Look for sequences of 0, apply logic from above to calc num new, add up across entire array, check boundaries
- [ ] quit early if count is >= N

#### Corner cases
empty sequences at start and end

In [ ]:
def max_insertable(seq_len: int) -> int:
    return int((seq_len-1)/2)


def can_place_flowers(flowerbed: list[int], n: int) -> bool:
    cnt, seq_len = 0, 0
    for p_idx in [0] + flowerbed + [0]:
        if p_idx == 0:
            seq_len += 1
        else:
            if seq_len > 0:
                cnt += max_insertable(seq_len)
                if cnt >= n:  # end as soon as possible
                    return True
            seq_len = 0
    cnt += max_insertable(seq_len)  # account for last sequence
    return cnt >= n

test_cases = [
    {"in": ([0,0], 1), "out": True},
    {"in": ([0,0], 2), "out": False},
    {"in": ([0,0,0], 2), "out": True},
    {"in": ([1,0,0,1], 1), "out": False},
    {"in": ([0,0,1,1,1,0], 2), "out": False},
    {"in": ([0,0,1,1,1,0], 1), "out": True},
]

run_tests(can_place_flowers, test_cases)


## Greatest Common String Divisor

### Problem Statement
For two strings s and t, we say "t divides s" if and only if s = t + ... + t (i.e., t is concatenated with itself one or more times).

Given two strings str1 and str2, return the largest string x such that x divides both str1 and str2.

In [ ]:
# This function is kind of the key idea ... defines divisor in an easy to calculate manner
def is_divisor(s: str, d: str) -> bool:
    return len(s)%len(d) == 0 and s == d*int(len(s)/len(d))

def string_gcd(str1: str, str2: str) -> str:
    # Look for longest divisor in the shorter of the two strings
    shorter, longer = sorted([str1,str2], key=len)
    # Handle empty string and full string match
    if len(shorter) == 0 or is_divisor(longer, shorter):
        return shorter
    # no need to go on for more than half since we need an integer multiple of length
    divisors = [shorter[0:i] for i in range(1,int(len(shorter)/2)+1) if is_divisor(shorter, shorter[0:i])]
    for d in sorted(divisors, key=len, reverse=True):
        if is_divisor(longer, d):
            return d
    return ""


In [ ]:
s = "ABCDEFGHIJK"
test_cases = [
    {"in": ("ABCABC", "ABC"), "out": "ABC"},
    {"in": ("ABABAB", "ABAB"), "out": "AB"},
    {"in": ("ABABABAB", "ABAB"), "out": "ABAB"},
    {"in": (s*100, s*2), "out": s*2},
    {"in": (s*101, s*2), "out": s},
    {"in": ("LEET", "CODE"), "out": ""},
]

run_tests(string_gcd, test_cases)


## Increasing Triplets

### Problem statement
Given an integer array nums, return true if there exists a triple of indices (i, j, k) such that i < j < k and nums[i] < nums[j] < nums[k]. If no such indices exists, return false.

Example 1:

Input: nums = [1,2,3,4,5]
Output: true
Explanation: Any triplet where i < j < k is valid.

**Analyze**<br>
- numbers do not have to be adjacent
- ascending order for 3 points

**Plan**<br>
- track smallest items of each that have been seen

(1,2)

see smaller element at index 0
store that
see smaller element at index 1 ... use that for True purposes

smallest_0
smallest_1 (that is larger than smallest 0)


walk over elements
- if current > smallest_1  RETURN TRUE
- smallest_1 = current if (current < smallest_1 and current > smallest_0) else smallest_1
- smallest_0 = min(smallest_0, current)

Return False

In [ ]:
nums = [1,2,3,4,5]

def increasingTriplet(nums: list[int]) -> bool:
    sm0, sm1 = 9e9,9e9
    for n in nums:
        if n > sm1:
            return True
        sm1 = n if (n < sm1 and n > sm0) else sm1
        sm0 = min(n, sm0)
    return False

## Reverse Vowels

In [ ]:
# s = "bob"

def reverseVowels(s: str) -> str:
    s = list(s)  # cannot do indexed assignment on a string
    indices = [i for i, v in enumerate(s) if v in "aeiouAEIOU"]
    for i in range(0, int(len(indices)/2)):
        j = -(i+1)
        tmp = s[indices[i]]
        s[indices[i]] = s[indices[j]]
        s[indices[j]] = tmp
    return "".join(s)


test_cases = [
    {"in": ("Bob",), "out": "Bob"},
    {"in": ("BooB",), "out": "BooB"},
    {"in": ("Bilbo",), "out": "Bolbi"},
    {"in": ("",), "out": ""},
    {"in": ("Boobi Joe",), "out": "Beobi Joo"},
    {"in": ("Boobi Joe Lu",), "out": "Buebo Jio Lo"},
]


run_tests(reverseVowels, test_cases)

## Move Zeroes

### Problem Statement
Given an integer array nums, move all 0's to the end of it while maintaining the relative order of the non-zero elements.

Note that you must do this in-place without making a copy of the array.


**Analyze**<br>
- in place
- maintain order

### Plan
move across the elements
If you find a 0, 

- Track the end of non-zero portion AND num zeros seen
- put non-zeros at end of nz portion, then at end fill rest with zeros (according to cnt)


In [ ]:
def move_zeroes(nums: list[int]) -> list[int]:
    nz_idx, z_cnt = 0, 0
    for n in nums:
        if n == 0:
            z_cnt += 1
        else:
            nums[nz_idx] = n
            nz_idx += 1
    if z_cnt:
        nums[-z_cnt:] = [0]*z_cnt

    return nums # remove when adding to leet code
    

test_cases = [
    {"in": ([0,1,0,3,12],), "out": [1,3,12,0,0]},
    {"in": ([0],), "out": [0]},
    {"in": ([1,2,3],), "out": [1,2,3]},
    {"in": ([1,0,0,0,0,2],), "out": [1,2,0,0,0,0]},
    {"in": ([2,0,3,0,0],), "out": [2,3,0,0,0]},
]

run_tests(move_zeroes, test_cases)

## Find Max moving average

### Problem Statement
Find a contiguous subarray, of length K, within an integer array (length >= k) that has the maximum average value and return this value.

**Analyze**<br>
Do not need to recalculate the average from scratch each time, just simply update the previous sum and then calculate avg (so 3 ops vs k+1 ops per LOOP)

In [ ]:
def find_max_average(nums: list[int], k: int) -> float:
        psum = sum(nums[0:k])
        mx = psum/k
        for i in range(1, 1+len(nums)-k):
            # reuse previous sum calculation for efficiency
            psum = (psum - nums[i-1]) + nums[(i+k)-1]
            mx = max(mx, psum/k)
        return mx

## Max Consecutive Ones

**Problem Statement**<br>
Given a binary array nums and an integer k, return the maximum number of consecutive 1's in the array if you can flip at most k 0's.


Example 1:

Input: nums = [1,1,1,0,0,0,1,1,1,1,0], k = 2
Output: 6
Explanation: [1,1,1,0,0,1,1,1,1,1,1]
Bolded numbers were flipped from 0 to 1. The longest subarray is underlined.
Example 2:

Input: nums = [0,0,1,1,0,0,1,1,1,0,1,1,0,0,0,1,1,1,1], k = 3
Output: 10
Explanation: [0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1]
Bolded numbers were flipped from 0 to 1. The longest subarray is underlined.


**Analyze**<br>
- can flip k 0's to 1 ...
- max consecutive
**corner cases**<br>
- All zeros vector
- Trailing and leading zeros with block of consecutive 1's in the middle
- All ones

**Key ideas**<br>
- Focus on which zeros to flip
- Create 2 new arrays
  - priors[i] contains the number of consec. 1's adjacent to i on left
  - afters[i] " .. to i on right

- Use sliding window caching concept from before for zero flips
  - Slide across sequential combos of 
  - Can use a sliding window as we know zero flips cannot be spread across an array, since we are looking for a block with consecutive ones and not the max number of consec. ones across the whole array
    - i.e. 1111011110000001101111 (k = 2) -> optimal is flipping first two 0 values, not the first and last 0 values (which would yield max sum of consecutive 1 patches, but not the largest single patch)


**Plan**<br>
<!-- - build priors and afters (O(n)) -->
- sliding windows through it
- Key idea is to reuse previous calculations, with minor additions and subtractions
  - Reuse calculations of # 1's prior in prior_cnt variable (resets on each 0)
  - Reuse calculations of # 1's after ...
    - define F(A,i,k) -> $n_0 , j_0$ as:
      - $n_0$ the number consecutive 1s occuring in A[i:], with k flips
      - $j_0$ the index of the 0 (within A) that ended the streak
    - Then we have:
      - F(A,i,k) = $n_0 , j_0$ is used for the first calculation
        - So we know # consec between i and $j_0$ is $n_0$.  So the number after a shift of 1 zero is: $n_0 - (i_1-i_0) + F(A,j_1,1)$
          - Number of 1s that were skipped when we moved to the new start ($i_0 - i_1$)
          - Number of 1s that are added when we flip zero at $j_1$, $F(A,j_1,1)$
          - Number of 1s between $i_0$ and $j_0$ with k flips, $n_0$

In [ ]:
def _streak(nums: list[int], idx0: int, k_flips: int) -> tuple[int,int]:
    """Find the number of consecutive 1's in nums, starting at idx0 and with k_flips"

    Args:
        nums (list[int]): complete list of integers
        idx0 (int): starting index for the search
        k_flips (int): number of 0->1 flips

    Returns:
        tuple[int,int]: Number of consecutive ones, index of 0 that ended streak
    """
    cnt = 0
    for j, n in enumerate(nums[idx0:]):
        if n == 1:
            cnt += 1
        elif k_flips > 0:
            cnt += 1
            k_flips -= 1
        else:
            break
    return cnt, j + idx0


def max_consec_ones(nums: list[int], k: int) -> int:
    mx, prior_cnt, after_cnt = 0, 0, None
    for i,n in enumerate(nums):
        if n == 1:
            prior_cnt += 1
        else:
            # streak if nums[i] and all consec 0s are switched to 1s
            if after_cnt is None:
                after_cnt, j = _streak(nums, i, k)
            else:
                tmp = _streak(nums, j, 1)
                after_cnt = after_cnt - (i - prev_i) + tmp[0]
                j = tmp[1]
            # print(i, prior_cnt, after_cnt, j)  # debug
            prev_i = i
            mx = max(mx, after_cnt+prior_cnt)
            prior_cnt = 0  # we hit a 0, so any streak that doesn't contain this flip is disconnected from previous values

    return max(prior_cnt, mx) # account for monotonic 1 vectors


test_cases = [
    {"in": ([1,1,1,0,0,0,1,1,1,1,0], 2), "out": 6},
    {"in": ([0,0,1,1,0,0,1,1,1,0,1,1,0,0,0,1,1,1,1], 3), "out": 10},
    {"in": ([1,1,0,1,1,0,1,1,0,0,0], 2), "out": 8},
    {"in": ([1,1,1,0,1,1,1,0,0,0,0,1,1,0,1,1], 2), "out": 8},
    {"in": ([0,0,0,0,0,0], 3), "out": 3},
    {"in": ([1,1,1,1], 3), "out": 4},
    {"in": ([0,0,1,1,1], 1), "out": 4},
    {"in": ([0,1,1,1,0], 2), "out": 5},
]

run_tests(max_consec_ones, test_cases)

# speed test
max_consec_ones(np.random.randint(0,2,size=int(10e4)) , 1500)

## Similar Strings

**Problem Statement**<br>
Two strings are considered close if you can attain one from the other using the following operations:

Operation 1: Swap any two existing characters.
For example, abcde -> aecdb
Operation 2: Transform every occurrence of one existing character into another existing character, and do the same with the other character.
For example, aacabb -> bbcbaa (all a's turn into b's, and all b's turn into a's)
You can use the operations on either string as many times as necessary.

Given two strings, word1 and word2, return true if word1 and word2 are close, and false otherwise.


**Analyze**<br>
op1 = string as sets are the same

lengths must be equal
string_as_sets must be equal
value counts (but not the keys) must be equal

In [ ]:
from collections import Counter

def fnc(w1: str, w2:str) -> bool:
    return len(w1) == len(w2) and set(w1) == set(w2) and Counter(Counter(w1).values()) == Counter(Counter(w2).values())


test_cases = [
    {"in": ("eabbccc","cabbbae"), "out": True},
    {"in": ("a","e"), "out": False},
    {"in": ("aaaaabbcccdddeeeeffff", "aaabbbbccddeeeeefffff"), "out": False},
    {"in": ("abc", "cba"), "out": True},
]

run_tests(fnc, test_cases)

## Equal Rows and Columns

Given a 0-indexed n x n integer matrix grid, return the number of pairs (ri, cj) such that row ri and column cj are equal.

A row and column pair is considered equal if they contain the same elements in the same order (i.e., an equal array).

### Analyze
- array equality
- hashable

Dump into a hashset, but keep track of count.  Then find intersection of set and add the associated counts together

- create frequency dict for rows and cols
  - iterate and use Counter
- find weighted intersection
  - for each k, cnt in set1, if k in set2, total_cnt += cnt * set2[k]

In [ ]:
from collections import Counter

def fnc(grid: list[list[int]]) -> int:
    count = 0
    row_freqs = Counter([tuple(r) for r in grid])
    col_freqs = Counter([tuple([r[col_idx] for r in grid]) for col_idx in range(len(grid))])

    for r, cnt in row_freqs.items():
        count += cnt * (col_freqs.get(r,0))
    return count

test_cases = [
    {"in": ([[3,2,1],[1,7,6],[2,7,7]],), "out": 1},
    {"in": ([[3,1,2,2],[1,4,4,5],[2,4,2,2],[2,4,2,2]],), "out": 3},
    {"in": ([[1]],), "out": 1},
]

run_tests(fnc, test_cases)

## Asteroids

We are given an array asteroids of integers representing asteroids in a row.

For each asteroid, the absolute value represents its size, and the sign represents its direction (positive meaning right, negative meaning left). Each asteroid moves at the same speed.

Find out the state of the asteroids after all collisions. If two asteroids meet, the smaller one will explode. If both are the same size, both will explode. Two asteroids moving in the same direction will never meet.


test cases
- all moving in same direction
- different directions

**plan**<br>
looking at pairs of these, with a value getting removed on collision

(i,j) collision? -> i > 0 & j < 0
on collision ->  remove min(abs(a[i]), abs[a[j]]) ... if a[j] was bigger, check if collision with neighbor to left.  Otherwise, check with neighbor to right

- we are done when we hit the end of the array

In [ ]:
def fnc(asteroids: list[int]) -> list[int]:
    i = 1
    while i < len(asteroids):
        if asteroids[i-1] > 0 and asteroids[i] < 0:  # collision check
            if asteroids[i-1] > abs(asteroids[i]):  # left asteroid wins, consider item to right
                asteroids.pop(i)
            elif asteroids[i-1] == abs(asteroids[i]):  # both explode, compare two on either side
                asteroids.pop(i)
                asteroids.pop(i-1)
                i = max(1, i-1)
            else:  # right asteroid is bigger, continue its destruction on left
                asteroids.pop(i-1)
                i = max(1, i-1)
        else:
            i += 1
    return asteroids


test_cases = [
    {"in": ([6, -1,  5, -5, -6],), "out": []},
    {"in": ([6, 5, -1],), "out": [6, 5]},
    {"in": ([1,-2,3,-4,5,-6,7],), "out": [-2,-4,-6,7]},
    {"in": ([-2,3],), "out": [-2,3]},
    {"in": ([10,-1,-1,-1,-1,-1,-1,-1,-1,-1],), "out": [10]},
    {"in": ([10,5,-6],), "out": [10]},
    {"in": ([8,-8],), "out": []},
    {"in": ([1,-1,-2,1],), "out": [-2,1]},
]

run_tests(fnc, test_cases)

## Decode String

Given an encoded string, return its decoded string.

The encoding rule is: k[encoded_string], where the encoded_string inside the square brackets is being repeated exactly k times. Note that k is guaranteed to be a positive integer.

You may assume that the input string is always valid; there are no extra white spaces, square brackets are well-formed, etc. Furthermore, you may assume that the original data does not contain any digits and that digits are only for those repeat numbers, k. For example, there will not be input like 3a or 2[4].

The test cases are generated so that the length of the output will never exceed 10e5.

Example 1:
Input: s = "3[a]2[bc]"
Output: "aaabcbc"

Example 2:
Input: s = "3[a2[c]]"
Output: "accaccacc"

Example 3:
Input: s = "2[abc]3[cd]ef"
Output: "abcabccdcdcdef"


**Plan**<br>

Push onto the stack until we hit a ], then pop from it until we hit the corresponding num and [.  Then create
the appropriate seq (subseq*num) and push that onto the stack, continue on

In [ ]:
def fnc(encoded: str) -> str:
    stack = []
    for s in encoded:
        if s == "]":
            seq, n = "", ""
            # start popping until you hit a [
            while stack[-1] != "[":
                seq = stack.pop() + seq  # order on stack is reversed, fix this
            stack.pop()  # remove the "["
            while stack and stack[-1].isdigit():
                n = stack.pop() + n  # order on stack is reversed, fix this
            stack.append("".join(seq) * int(n))  # push back on
        else:
            stack.append(s)

    return "".join(stack)




test_cases = [
    {"in": ("3[a]2[bc]",), "out": "aaabcbc"},
    {"in": ("3[a2[c]]",), "out": "accaccacc"},
    {"in": ("2[abc]3[cd]ef",), "out": "abcabccdcdcdef"},
    {"in": ("10[leetcode]",), "out": "leetcode"*10},
]

run_tests(fnc, test_cases)